In [1]:
!git clone https://github.com/saahiluppal/catr.git

fatal: destination path 'catr' already exists and is not an empty directory.


In [2]:
import os
os.chdir("./catr")
os.getcwd()
!pip install -r requirements.txt
os.chdir("../")
os.getcwd()


'c:\\Users\\takah\\Desktop\\python_files\\bokete'

In [3]:
import pandas  as pd
import numpy as np
import io
from PIL import Image
import zipfile

In [4]:
import torch

from transformers import BertTokenizer
from PIL import Image
import argparse

from catr.models import caption
from catr.datasets import coco, utils
from catr.configuration import Config
import os

# parser = argparse.ArgumentParser(description='Image Captioning')
# parser.add_argument('--path', type=str, help='path to image', required=True)
# parser.add_argument('--v', type=str, help='version', default='v3')
# parser.add_argument('--checkpoint', type=str, help='checkpoint path', default=None)
# args = parser.parse_args()
# image_path = args.path
# version = args.v
# checkpoint_path = args.checkpoint
class catr_model:
    def __init__(self):
        version = "v3"
        checkpoint_path = None

        self.config = Config()

        if version == 'v1':
            self.model = torch.hub.load('saahiluppal/catr', 'v1', pretrained=True)
        elif version == 'v2':
            self.model = torch.hub.load('saahiluppal/catr', 'v2', pretrained=True)
        elif version == 'v3':
            self.model = torch.hub.load('saahiluppal/catr', 'v3', pretrained=True)
        else:
            print("Checking for checkpoint.")
            if checkpoint_path is None:
                raise NotImplementedError('No model to chose from!')
            else:
                if not os.path.exists(checkpoint_path):
                    raise NotImplementedError('Give valid checkpoint path')
                print("Found checkpoint! Loading!")
                model,_ = caption.build_model(self.config)
                print("Loading Checkpoint...")
                checkpoint = torch.load(checkpoint_path, map_location='cpu')
                model.load_state_dict(checkpoint['model'])
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        self.start_token = self.tokenizer.convert_tokens_to_ids(self.tokenizer._cls_token)
        self.end_token = self.tokenizer.convert_tokens_to_ids(self.tokenizer._sep_token)

    def fit(self,image_path):
        self.image = Image.open(image_path)
        self.image = coco.val_transform(self.image)
        self.image = self.image.unsqueeze(0)

        self.caption, self.cap_mask = self.create_caption_and_mask(
        self.start_token, self.config.max_position_embeddings)

        output = self.evaluate()
        result = self.tokenizer.decode(output[0].tolist(), skip_special_tokens=True)
        #result = tokenizer.decode(output[0], skip_special_tokens=True)
        return (result.capitalize())

    def create_caption_and_mask(self,start_token, max_length):
        caption_template = torch.zeros((1, max_length), dtype=torch.long)
        mask_template = torch.ones((1, max_length), dtype=torch.bool)

        caption_template[:, 0] = start_token
        mask_template[:, 0] = False

        return caption_template, mask_template

    @torch.no_grad()
    def evaluate(self):
        self.model.eval()
        for i in range(self.config.max_position_embeddings - 1):
            predictions = self.model(self.image, self.caption, self.cap_mask)
            predictions = predictions[:, i, :]
            predicted_id = torch.argmax(predictions, axis=-1)

            if predicted_id[0] == 102:
                return self.caption

            self.caption[:, i+1] = predicted_id[0]
            self.cap_mask[:, i+1] = False

        return self.caption

c:\Users\takah\anaconda3\envs\test_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
imcap = catr_model()

Using cache found in C:\Users\takah/.cache\torch\hub\saahiluppal_catr_master
c:\Users\takah\anaconda3\envs\test_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\takah\anaconda3\envs\test_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
imcap.fit("../data/bokete/train/00fx15srqs.jpg")

C:\Users\takah/.cache\torch\hub\saahiluppal_catr_master\models\position_encoding.py:38: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


A girl is holding a colorful umbrella in her hand.


In [ ]:
mask_template = torch.ones((1, max_length), dtype=torch.bool)

caption_template[:, 0] = start_token

In [ ]:
train_df = pd.read_csv("../data/bokete/train.csv")
test_df = pd.read_csv("../data/bokete/test.csv")


In [ ]:
# zipファイルのパス
zip_path = '../data/bokete/train.zip'

# 配列格納用のList
train_imgs = []

# zipの読み込み
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # zipファイル内の各ファイルについてループ
    for info in zip_file.infolist():
        # 「zipファイル名/」については処理をしない
        if (info.filename != 'data/'):
            # 対象の画像ファイルを開く
            with zip_file.open(info.filename) as img_file:
                # 画像のバイナリデータを読み込む
                img_bin = io.BytesIO(img_file.read())
                # バイナリデータをpillowから開く
                img = Image.open(img_bin)
                # 画像データを配列化
                img_array = np.array(img)
                # 格納用のListに追加
                train_imgs.append(img_array)

# 処理が完了後、np.arrayに変換
train_imgs = np.array(train_imgs)

In [ ]:
# zipファイルのパス
zip_path = '../data/bokete/test.zip'

# 配列格納用のList
test_imgs = []

# zipの読み込み
with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # zipファイル内の各ファイルについてループ
    for info in zip_file.infolist():
        # 「zipファイル名/」については処理をしない
        if (info.filename != 'data/'):
            # 対象の画像ファイルを開く
            with zip_file.open(info.filename) as img_file:
                # 画像のバイナリデータを読み込む
                img_bin = io.BytesIO(img_file.read())
                # バイナリデータをpillowから開く
                img = Image.open(img_bin)
                # 画像データを配列化
                img_array = np.array(img)
                # 格納用のListに追加
                test_imgs.append(img_array)

# 処理が完了後、np.arrayに変換
test_imgs = np.array(test_imgs)

In [ ]:
import torch
import numpy as np
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from tasks.mm_tasks.refcoco import RefcocoTask
 
from models.ofa import OFAModel
from PIL import Image
 
import cv2
import numpy
 
tasks.register_task('refcoco', RefcocoTask)
 
# turn on cuda if GPU is available
use_cuda = torch.cuda.is_available()
# use fp16 only when GPU is available
use_fp16 = False
 
# specify some options for evaluation
parser = options.get_generation_parser()
input_args = ["", "--task=refcoco", "--beam=10", "--path=checkpoints/ofa_large.pt", "--bpe-dir=utils/BPE"]
args = options.parse_args_and_arch(parser, input_args)
cfg = convert_namespace_to_omegaconf(args)